In [48]:
# We'll first need the data from my drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
!pip install chatterbot -qq
!pip install chatterbot-corpus -qq
!python -m spacy download en_core_web_sm -qq

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 5.7MB 7.0MB/s 
     |████████████████████████████████| 204kB 48.3MB/s 
     |████████████████████████████████| 30.9MB 142kB/s 
     |████████████████████████████████| 235kB 53.1MB/s 
     |████████████████████████████████| 266kB 49.8MB/s 
     |████████████████████████████████| 3.2MB 35.2MB/s 
     |████████████████████████████████| 2.1MB 43.7MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
ERROR: fbprophet 0.6 has requirement python-dateutil>=2.8.0, but you'll have python-dateutil 2.7.5 which is incompatible.
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 276kB 9.1MB/s 
ERROR: chatterbot 1.0.5 has requirement 

In [49]:
path='/content/drive/My Drive/1000ml/Project 7 - Chatbot/Data/'
import colab_env
import os

# We'll need the following nltk packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# We'll use lemmas instead of stems.
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# We will tag the parts of the speech 
from nltk.tag import pos_tag

# Get the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

# We'll need json to open a file of intents
import json

# We will need to save the model
import pickle

# Some helper libraries
import numpy as np
import pandas as pd
import random
import re

# We'll need these to actually create a machine learning model.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adagrad, Adam, Adadelta

# We may need to try out a grid search
from sklearn.model_selection import GridSearchCV

# We'll need this for the in-code tagging of words
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import Matcher

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import Data and Clean

In [50]:
# first we'll need the pairs of conversation:
data_file1 = open(f'{path}frames.json').read()
travel_txt = json.loads(data_file1)

In [51]:
# Let's make a list of lists for the chatbot to train with. Each list is a conversation.
conversations = []
for convo in travel_txt:
  turns = []
  for turn in convo['turns']:
    turns.append(turn['text'])
  conversations.append(turns)

In [5]:
# # These files contain a whole bunch of conversation type text, which are from internet forums
# data_file2 = pd.read_csv(f'{path}dialogueText.csv')
# data_file3 = pd.read_csv(f'{path}dialogueText_196.csv')
# data_file4 = pd.read_csv(f'{path}dialogueText_301.csv')

# # First require that everything in each row is a string.
# data_file2['text'] = data_file2['text'].astype(str)
# data_file3['text'] = data_file3['text'].astype(str)
# data_file4['text'] = data_file4['text'].astype(str)

# # We'll split all the sentences of each dialogue with the bar '|' since its unlikely to be in text. 
# df2 = data_file2.groupby('dialogueID')['text'].agg(lambda x: '|'.join(x)).reset_index()
# df3 = data_file3.groupby('dialogueID')['text'].agg(lambda x: '|'.join(x)).reset_index()
# df4 = data_file4.groupby('dialogueID')['text'].agg(lambda x: '|'.join(x)).reset_index()

# # Now split the strings on the bar, to make a list out of each element.
# df2['text']=df2['text'].str.split('\|')
# df3['text']=df3['text'].str.split('\|')
# df4['text']=df4['text'].str.split('\|')

# # Now that the cleaned dfs are created, we'll delete the old ones to clear up RAM
# del data_file2
# del data_file3
# del data_file4

### A few extra hard coded conversations

In [52]:
# Greetings
g1 = ['Hello!',
      'Hi there']
g2 = ['Hi there',
      'Hello!']
g3 = ['Greetings',
      'Hey']
g4 = ['Hey',
      'Oh hey!']
g5 = ['Good morning',
      'Good morning']
g6 = ['Good evening',
      'Good evening']

greetings = [g1,g2,g3,g4,g5,g6]

# Farewells
f1 = ['Bye','Goodbye!']
f2 = ['See ya','Good bye!']
f3 = ['Have a good night','You as well, goodbye!']
f4 = ['Farewell','Later!']
f5 = ['Goodbye!', 'Bye for now!']

farewells = [f1,f2,f3,f4,f5]

In [53]:
convo1=["I would like to book a flight.",
        "I can help you with that. Where are you traveling to",
        "I am traveling to Singapore.",
        "What date will you be traveling?",
        "I want to fly on June 14th.",
        "Do you want to fly out of Los Angeles International or Burbank Airport?",
        "I would like to fly out of Los Angeles International Airport.",
        "Would you prefer a morning or an afternoon flight?",
        "I would rather fly in the morning.",
        "Well, I have you booked on a flight that will fit your schedule. The tickets will arrive by mail in a few days."]

convo2=['I want to buy a plane ticket.',
        'I can help you make your reservation. What is your destination?',
        'My final destination is Sydney, Australia.',
        'What is your travel date?',
        'I am making a reservation for December 12th.',
        'Would you prefer to leave from Los Angeles International or Burbank Airport?',
        'I want to fly out of Burbank Airport.'
        'Would you rather fly in the morning or later in the day?',
        'I need an afternoon flight.',
        'I have you booked on a flight in the afternoon. Your tickets will arrive in the mail within 3 days.']

convo3 =['I need to make a plane reservation.',
         'We can book your trip right now. What city are you flying to?',
         'I need to fly to New York City.',
         'What date would you like me to book this plane ticket for?',
         'I need a flight on July 4th.',
         'You can fly out of Los Angeles International or Burbank Airport. Which do you prefer?',
         'I will fly out of whatever airport has the cheapest price.',
         'If you have a choice, what time of day would you prefer to fly?',
         'I would like to be booked on the least expensive flight.',
         'I was able to find an inexpensive flight for you. We are sending you your tickets in the mail.']

convos = [convo1,convo2,convo3]

# Training!

In [55]:
# Now we have a bunch of conversational data that we can use to train the chatbot. Let's see how well it works.
chatbot = ChatBot('Traveller',logic_adapters=["chatterbot.logic.BestMatch"])
travel_listtrainer = ListTrainer(chatbot)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
for convo in conversations:
  travel_listtrainer.train(convo)

In [57]:
for convo in convos:
  travel_listtrainer.train(convo)

for greet in greetings:
  travel_listtrainer.train(greet)

for bye in farewells:
  travel_listtrainer.train(bye)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [56]:
travel_corpustrainer = ChatterBotCorpusTrainer(chatbot)

In [58]:
travel_corpustrainer.train("chatterbot.corpus.english")

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [77]:
chatbot.generate_response(list('I want to book a flight'))

AttributeError: ignored

In [32]:
chatbot.get_initialization_functions()

{<function chatterbot.utils.download_nltk_averaged_perceptron_tagger>,
 <function chatterbot.utils.download_nltk_stopwords>,
 <function chatterbot.utils.download_nltk_wordnet>}

# Some NER functions

For this section, we need some functions that will take in a string, and pull out relevant information. We will make some assumptions here. We need the following information:
* Origin Date
* Destination Date
* Origin City
* Destination City
* Budget 
* Number of people

Since both number of people and budget will be numbers, we will assume that the budget will always be greater than the number of people. The only issue with that is if the numbers are written as words and the program can't parse them correctly. 

## Dates

In [191]:
# Start with a function that extracts dates
import dateutil.parser as dparser

# We'll note here that I'll have to consider what happens when the bot asks for the second date. Does it run through this again, or do I create a second function. 
# This will have to be changed depending on the global variables I choose to inlcude

def get_dates(message):
  '''
  This function checks if there are any dates in the message, if so it pulls them out. If there are two dates, then we need to
  parse the list more closely to pull out each date. This function assumes that the date(s) was/were written in a readable form.
  '''
  # Remove punctuation
  clean_message = re.sub(r'[^\w\s]','',message)
  # Next, check if there is only one date. If so, we will assume that this is the date they want to leave on.
  try:
    depart_date = dparser.parse(clean_message,fuzzy=True)
    return_date = np.datetime64('NaT')
  except ValueError:
    # I will also assume here that the second date given is the return date
    # As a final assumption, we assume that there are only two dates given maximum.
    # Let's first recognize the positions of the dates in the string.
    matcher = Matcher(nlp.vocab)
    matcher.add("DateFinder", None, [{'LOWER':'on'}, {"ENT_TYPE": "DATE"}],
                                    [{'LOWER':'from'}, {"ENT_TYPE": "DATE"}],
                                    [{'LOWER':'to'}, {"ENT_TYPE": "DATE"}],
                                    [{'IS_STOP':'TRUE'}, {"ENT_TYPE": "DATE"}])
    matches = matcher(nlp(clean_message))
    # First and second date index matchings
    d1 = matches[0]
    d2 = matches[1]
    # Go from first part of date, to one word after (This gets the year, unless there is none, in which case it gets a filler word)
    first_date = nlp(message)[d1[1]:d1[2]+2]
    # If the message ends with the second date, then we need to be careful to not pick up an error
    if (d2[2]==len(clean_message) or d2[2]+1==len(clean_message) or d2[2]+2==len(clean_message)):
      second_date = nlp(clean_message)[d2[1]:]
    else:
      second_date = nlp(clean_message)[d2[1]:d2[2]+2]
    depart_date = dparser.parse(str(first_date),fuzzy=True)
    return_date = dparser.parse(str(second_date),fuzzy=True)
  
  dates = (depart_date, return_date)

  return dates

### Tests

In [192]:
string = 'I want to book a trip from Toronto to England. I will leave on March 2, 2020 and return on august 1, 2020'
get_dates(string)

(datetime.datetime(2020, 3, 2, 0, 0), datetime.datetime(2020, 8, 1, 0, 0))

In [193]:
string2 = "book me a flight from toronto to another place from March 8th, 2020 to "
get_dates(string2)

(datetime.datetime(2020, 3, 8, 0, 0), numpy.datetime64('NaT'))

In [194]:
# there is a required order
string3 = 'I gotta go to a new place man. Can I leave on the first of March 2020?'
get_dates(string3)

(datetime.datetime(2020, 3, 24, 0, 0), numpy.datetime64('NaT'))

In [197]:
# does not work with word numbers.
string4 = "book me a flight from toronto to another place from April first, 2020 to "
get_dates(string4)

(datetime.datetime(2020, 4, 24, 0, 0), numpy.datetime64('NaT'))

## Locations

In [217]:
def get_locs(message):
  # Remove punctuation
  clean_message = re.sub(r'[^\w\s]','',message)

  from_matcher = Matcher(nlp.vocab)
  from_matcher.add("LocFinder", None, [{'LOWER':'from'},{'ENT_TYPE':'GPE'}],
                                      [{'LOWER':'of'},{'ENT_TYPE':'GPE'}])
  to_matcher = Matcher(nlp.vocab)
  to_matcher.add("LocFinder", None, [{'LOWER':'to'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'visit'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'see'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'out'},{'ENT_TYPE':'GPE'}],
                                    [{'LOWER':'travel'},{'ENT_TYPE':'GPE'}])
  from_match = from_matcher(nlp(clean_message))
  to_match = to_matcher(nlp(clean_message))
  if len(from_match)>0:
    loc1 = from_match[0]
    origin = str(nlp(clean_message)[loc1[1]+1])
  if len(to_match)>0:
    loc2 = to_match[0]
    dest = str(nlp(clean_message)[loc2[1]+1])
  
  locations = (origin, dest)

  return locations


In [218]:
string = 'I want to book a trip out of Toronto to England. I will leave on March 2, 2020 and return on august 1, 2020'
get_locs(string)

('Toronto', 'England')

In [208]:
pip install amadeus

     |████████████████████████████████| 61kB 1.9MB/s 


In [211]:
from amadeus import Client, ResponseError

amadeus = Client(
    client_id=os.getenv("AMADEUS_API_KEY"),
    client_secret=os.getenv("AMADEUS_SECRET")
)

try:
    response = amadeus.shopping.flight_offers_search.get(
        originLocationCode='SYD',
        destinationLocationCode='BKK',
        departureDate='2020-10-01',
        returnDate='2020-10-10',
        maxPrice=2000,
        currencyCode='CAD',
        adults=1)
    print(response.data)
except ResponseError as error:
    print(error)

[{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2020-10-01', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT14H35M', 'segments': [{'departure': {'iataCode': 'SYD', 'terminal': '1', 'at': '2020-10-01T22:10:00'}, 'arrival': {'iataCode': 'TPE', 'terminal': '2', 'at': '2020-10-02T05:40:00'}, 'carrierCode': 'CI', 'number': '52', 'aircraft': {'code': '359'}, 'operating': {'carrierCode': 'CI'}, 'duration': 'PT9H30M', 'id': '44', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'TPE', 'terminal': '1', 'at': '2020-10-02T07:05:00'}, 'arrival': {'iataCode': 'BKK', 'at': '2020-10-02T09:45:00'}, 'carrierCode': 'CI', 'number': '833', 'aircraft': {'code': '359'}, 'operating': {'carrierCode': 'CI'}, 'duration': 'PT3H40M', 'id': '45', 'numberOfStops': 0, 'blacklistedInEU': False}]}, {'duration': 'PT14H15M', 'segments': [{'departure': {'iataCode': 'BKK', 'at': '2

It should be noted that my next step is to pull out a budget and a number of people. Beyond that, I'll need to figure out how to turn a city into an Airport code. If someone says they want to go to Toronto, what airport should they use? But also, if someone says they want to go to Caledon, what airport should they use. 

In [11]:
city_code = pd.read_csv(f'{path}Airports.csv').groupby('Location').max()
code_dict = city_code.to_dict()['Code']

In [14]:

code_dict

{'Atlanta': 'ATL',
 'Barcelona': 'BCN',
 'Beijing': 'PEK',
 'Changi': 'SIN',
 'Chicago': 'ORD',
 'Dallas': 'DFW',
 'Delhi': 'DEL',
 'Denver': 'DEN',
 'Dominica': 'DOM',
 'Dubai': 'DXB',
 'France': 'CDG',
 'Frankfurt': 'FRA',
 'Guangdong': 'CAN',
 'Holland': 'AMS',
 'Istanbul': 'IST',
 'Las Vegas': 'LAS',
 'London': 'LHR',
 'Los Angeles': 'LAX',
 'Madrid': 'MAD',
 'Mexico City': 'MEX',
 'New York': 'JFK',
 'Shanghai': 'PVG',
 'Sydney': 'SYD',
 'Tokyo': 'HND',
 'Toronto': 'YYZ',
 'Washington': 'SEA'}

In [16]:
def iata_code_lookup(city, codes, origin_city=True):
  '''
  This function takes in a city name as well as a dictionary with city names and their IATA codes. It then checks whether this is the origin city or not.
  If the city is in the dictionary, it will output the code. Otherwise, it will output an empty string and an error message that depends on whether or not the city was the origin or destination.
  '''
  error = ''
  try: 
    iata_code = codes['city']
  except KeyError:
    iata_code = ''
    if origin_city:
      error = f'Unfortunately there are no flights leaving {city} at this time.'
    else:
      error = f'Unfortunately there are no flights to {city} at this time.'
  return (iata_code, error)